# Brute Forces, Secretaries, and Dichotomies

* Chapter 11 of [Real World Algorithms](https://mitpress.mit.edu/books/real-world-algorithms).

---

> Panos Louridas<br />
> Athens University of Economics and Business

## Sequential Search

* Sequential search is perhaps the most straightforward search method.

* We start from the beginning and check each item in turn until we find the one we want.

* It can be used for both sorted and unsorted data, but there are much better methods for sorted data.

* Here is a straightforward implementation:

In [1]:
def sequential_search(a, s):
    for i, element in enumerate(a):
        if element == s:
            return i
    return -1

* Let's check it on a random list:

In [2]:
import random

a = list(range(1000))
random.shuffle(a)

pos = sequential_search(a, 314)
print(a[pos], 'found at', pos)

pos = sequential_search(a, 1001)
print(pos)

314 found at 166
-1


* We need not write `sequential_search(a, s)` in Python.

* If `a` is a list, we can use `a.index(s)` instead.

* In fact that's what we should do, because it is way faster (we saw that also in Chapter 7).

* Here is the timing for our version:

In [3]:
import timeit

total_elapsed = 0
for i in range(100):
    a = list(range(10000))
    random.shuffle(a)
    start = timeit.default_timer()
    index = sequential_search(a, 314)
    end = timeit.default_timer()
    total_elapsed += end - start
print(total_elapsed)

0.03652806024183519


* And here is the timing for the native version (which actually calls a function written in C):

In [4]:
total_elapsed = 0
for i in range(100):
    a = list(range(10000))
    random.shuffle(a)
    start = timeit.default_timer()
    index = a.index(314)
    end = timeit.default_timer()
    total_elapsed += end - start

print(total_elapsed)

0.014140260143904015


# Matching, Comparing, Records, Keys

* When we are searching for an item in the list, Python performs an equality test between each item and the item we are searching for.

* The equality test is performed with the operator `==`.

* Checking for equality is not the same as checking whether two items are the *same*.

* This is called `identity checking` and in Python it is implemented with the operator `is`.

* That means that the following two are equal:

In [5]:
an_item = (1, 2)

another_item = (1, 2)

an_item == another_item

True

* But they are not the same:

In [6]:
an_item is another_item

False

* As another example, let's see what happens with Python's support for complex numbers:

In [7]:
x = 3.14+1.62j
y = 3.14+1.62j
print(x == y)
print(x is y)

True
False


* Identity checking is must faster than equality checking, but it is not what we usually want to use.

* A common idiom for identity checking in Python is checking for `None`, like `if a is None` or `if a is not None`.

* In many cases, we hold information for entities in *records*, which are collections of *attributes*.

* In that case, we want to search for an entity based on a particular attribute that identifies it.

* The attribute is called a *key*.

* In Python we can represent records as *objects* that are instances of a class.

* Alternatively, we can represent them as dictionaries.

* In fact, Python objects use dictionaries internally.

* Let's get a list of two persons.

In [8]:
john = {
    'first_name': 'John',
    'surname': 'Doe',
    'passport_no': 'AI892495',
    'year_of_birth': 1990,
    'occupation': 'teacher'
}

jane = {
    'first_name': 'Jane',
    'surname': 'Roe',
    'passport_no': 'AI485713',
    'year_of_birth': 1986,
    'occupation': 'civil engineer'
}

persons = [john, jane]

* In this example, the key for our search would be the passport number, because we would like to find the full information for a person with that particular piece of identification.

* To do that we could re-implement sequential search so that we provide to it the comparison function.

In [9]:
def sequential_search_m(a, s, matches):
    for i, element in enumerate(a):
        if matches(element, s):
            return i
    return -1

def match_passport(person, passport_no):
    return person['passport_no'] == passport_no

pos = sequential_search_m(persons, 'AI485713', match_passport)
print(persons[pos], 'found at', pos)

{'first_name': 'Jane', 'surname': 'Roe', 'passport_no': 'AI485713', 'year_of_birth': 1986, 'occupation': 'civil engineer'} found at 1


* Although you would probably use something more Pythonic like:

In [10]:
results = [(i, p) for i, p in enumerate(persons) if p['passport_no'] == 'AI485713']
results

[(1,
  {'first_name': 'Jane',
   'occupation': 'civil engineer',
   'passport_no': 'AI485713',
   'surname': 'Roe',
   'year_of_birth': 1986})]

## Self-Organizing Search

* In self-organizing search, we take advantage of an item's popularity to move it to the front of the collection in which we are performing our searches.

* In the move-to-front method, when we find an item we move it directly to the front.

* In the transposition method, when we find an item we swap it with its predecessor (if any).

* We cannot implement directly the algorithms for lists given in the book (that is, algorithm 11.2 and algorithm 11.3) for the simple reason that Python hides the list implementation from us.

* Moreover, Python lists are *not* linked lists. They are variable-length arrays (see the online documentation for details on the [implementation of lists in Python](https://docs.python.org/3/faq/design.html#how-are-lists-implemented)).

* We can implement algorithm 11.3, which is the transposition method for arrays.

In [11]:
def transposition_search(a, s):
    for i, item in enumerate(a):
        if item == s:
            if i > 0:
                a[i-1], a[i] = a[i], a[i-1]
                return i-1
            else:
                return i
    return -1

* How can we test `transposition_search(a, s)`?

* We need to do some groundwork to emulate a situation of popularity-biased searches.

* In particular, we will create a setting where the items we are searching for are governed by Zipf's law.

* First, we'll write a function that provides the Zipf probability for $n$ items.

In [12]:
def zipf(n):
    h = 0
    for x in range(1, n+1):
        h += 1/x
    z = [ 1/x * 1/h for x in range(1, n+1) ]
    return z

* We'll work with 1000 items:

In [13]:
zipf_1000 = zipf(1000)

* We can check that they sum up to 1, and see the first 20 of the probabilities.

In [14]:
print(sum(zipf_1000))
print(zipf_1000[:20])

1.0000000000000018
[0.13359213049244018, 0.06679606524622009, 0.044530710164146725, 0.033398032623110044, 0.026718426098488037, 0.022265355082073363, 0.019084590070348597, 0.016699016311555022, 0.014843570054715576, 0.013359213049244019, 0.012144739135676381, 0.011132677541036681, 0.010276317730187707, 0.009542295035174298, 0.008906142032829346, 0.008349508155777511, 0.007858360617202364, 0.007421785027357788, 0.007031164762760009, 0.006679606524622009]


* Again we will be performing our searches on 1000 items, in random order.

In [15]:
a = list(range(1000))
random.shuffle(a)
print(a[:20])

[824, 941, 606, 334, 421, 531, 899, 856, 714, 752, 389, 721, 749, 56, 196, 182, 787, 705, 937, 780]


* We will perform 1000 searches among these items.

* We want the searches to follow Zipf's law.

* First, we will create another list of 1000 items in random order.

In [16]:
b = list(range(1000))
random.shuffle(b)
print(b[:20])

[331, 748, 197, 270, 978, 46, 134, 29, 899, 21, 802, 925, 454, 234, 230, 147, 37, 121, 453, 119]


* Then, we will select 100000 items from the second list, using the Zipf probabilities.

* That means that we will be selecting the first item with probability `zipf_1000[0]`, the second item with probability `zipf_1000[1]`, and so on.


In [17]:
searches = random.choices(b, weights=zipf_1000, k=100000)

* Indeed, we can verify that the popularity of items in `searches` mirrors `b`:

In [18]:
from collections import Counter

counted = Counter(searches)
counted.most_common(20)


[(331, 13389),
 (748, 6737),
 (197, 4464),
 (270, 3270),
 (978, 2606),
 (46, 2291),
 (134, 1826),
 (29, 1739),
 (899, 1563),
 (21, 1274),
 (802, 1219),
 (925, 1137),
 (454, 1052),
 (234, 910),
 (230, 893),
 (147, 882),
 (37, 788),
 (121, 726),
 (453, 710),
 (119, 668)]

* So, we will perform 100000 searches in the first list, using as keys the items in `searches`.

* Because `transposition_search(a, s)` changes `a`, we will keep a copy of it to use it to compare the performance with simple sequential search.

* At the end, apart from displaying the time elapsed we will also show the first items of the changed `a`, to see how popular searches have gone to the beginning.

In [19]:
a_copy = a[:]

total_elapsed = 0
for s in searches:
    start = timeit.default_timer()
    index = transposition_search(a, s)
    end = timeit.default_timer()
    total_elapsed += end - start

print(total_elapsed)
print(a[:20])

1.8748526353447232
[748, 331, 270, 197, 978, 29, 899, 46, 147, 134, 802, 21, 234, 925, 230, 121, 454, 934, 158, 716]


* We will now perform the same searches with `a_copy` using simple sequential search.

In [21]:
total_elapsed = 0
for s in searches:
    start = timeit.default_timer()
    index = sequential_search(a_copy, s)
    end = timeit.default_timer()
    total_elapsed += end - start

print(total_elapsed)

3.336524669721257


# The Secretary Problem